# 📊 IBI Data Science Internship - Final Analysis
This notebook includes full code for customer segmentation, churn prediction, sales forecasting, and market basket analysis.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from statsmodels.tsa.arima.model import ARIMA
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


## 1. Customer Segmentation (RFM + KMeans)

In [ ]:
# Simulated customer RFM data
data = pd.DataFrame({
    'CustomerID': range(1, 1001),
    'Recency': np.random.randint(10, 365, 1000),
    'Frequency': np.random.randint(1, 50, 1000),
    'Monetary': np.random.randint(100, 1000, 1000)
})
data['CLV'] = data['Frequency'] * data['Monetary']
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(data[['Recency', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=4, random_state=42)
data['Segment'] = kmeans.fit_predict(rfm_scaled)
data.head()

## 2. Churn Prediction (Random Forest)

In [ ]:
# Simulated churn dataset
from sklearn.model_selection import train_test_split

data['Churn'] = np.random.choice([0, 1], size=len(data), p=[0.8, 0.2])
X = data[['Recency', 'Frequency', 'Monetary', 'CLV']]
y = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_scores = model.predict_proba(X_test)[:, 1]

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_scores))

## 3. Sales Forecasting (ARIMA)

In [ ]:
# Simulated sales data
date_range = pd.date_range(start="2022-01-01", end="2024-12-31", freq="D")
sales_data = pd.DataFrame({
    'ds': date_range,
    'y': np.random.normal(loc=5000, scale=1000, size=len(date_range)).clip(min=0)
})
monthly_sales = sales_data.resample('M', on='ds').sum().reset_index()
model_arima = ARIMA(monthly_sales['y'], order=(1, 1, 1))
model_fit = model_arima.fit()
forecast = model_fit.forecast(steps=6)
print(forecast)

## 4. Market Basket Analysis (Apriori)

In [ ]:
# Simulated transactions
transactions = [
    ['Bread', 'Milk'],
    ['Bread', 'Diaper', 'Beer', 'Eggs'],
    ['Milk', 'Diaper', 'Beer', 'Coke'],
    ['Bread', 'Milk', 'Diaper', 'Beer'],
    ['Bread', 'Milk', 'Diaper', 'Coke'],
]
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_basket = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df_basket, min_support=0.4, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules.head()